In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
data = pd.read_csv('''C:\\Users\\user\\Downloads\\a_steam_data_2021_2025.csv''')

In [ ]:
data.head

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.histplot(data['price'], kde=True)
plt.show()
sns.histplot(data['recommendations'], kde=True)
plt.show()
sns.histplot(data['log_price'], kde=True)
plt.show()
sns.histplot(data['log_recommendations'], kde=True)
plt.show()

In [ ]:
data['release_date'] = pd.to_datetime(data['release_date'], format='mixed', errors='coerce')
data['release_date'] = data['release_date'].fillna(pd.to_datetime('Aug 1 ' + data['release_year'].astype(str), format='%b %d %Y', errors='coerce'))       
data['genres'] = data['genres'].astype(str).str.split(';')
data['categories'] = data['categories'].astype(str).str.split(';')

In [ ]:
data['log_price'] = np.log1p(data['price'])
data['log_recommendations'] = np.log1p(data['recommendations'])

In [ ]:
data['is_hit'] = data['recommendations'] > 800
#data['is_mega_hit'] = data['recommendations'] > 1000
data['is_discount'] = (data['price'] % 1) > 0.90
data['developer_hit_ratio'] = data.copy().sort_values('release_date').groupby('developer')['is_hit'].transform(lambda x: x.shift(1).expanding().mean())
data['developer_hit_ratio'] = data['developer_hit_ratio'].fillna(0)
data['publisher_hit_ratio'] = data.copy().sort_values('release_date').groupby('publisher')['is_hit'].transform(lambda x: x.shift(1).expanding().mean())
data['publisher_hit_ratio'] = data['publisher_hit_ratio'].fillna(0)
data['developer_game_number'] = data.copy().sort_values('release_date').groupby('developer').cumcount() + 1
data['publisher_game_number'] = data.copy().sort_values('release_date').groupby('publisher').cumcount() + 1
data['timeliness'] = data['recommendations'] / ((2026 - data['release_date'].astype(str).str.split('-').str[0].astype(int)) * 12 - (data['release_date'].astype(str).str.split('-').str[1].astype(int) - 1)).replace(0, 1)
data['release_month'] = data['release_date'].dt.month
data['month_sin'] = np.sin(2 * np.pi * data['release_month'] / 12)
data['month_cos'] = np.cos(2 * np.pi * data['release_month'] / 12)

data['developer_genre_overlap'] = [len(set(g) & set(h)) / len(set(g)) if len(g) > 0 else 0 for g, h in zip(data['genres'], data['developer_genres'])]
data['developer_category_overlap'] = [len(set(c) & set(h)) / len(set(c)) if len(c) > 0 else 0 for c, h in zip(data['categories'], data['developer_categories'])]
data['publisher_genre_overlap'] = [len(set(g) & set(h)) / len(set(g)) if len(g) > 0 else 0 for g, h in zip(data['genres'], data['publisher_genres'])]
data['publisher_category_overlap'] = [len(set(c) & set(h)) / len(set(c)) if len(c) > 0 else 0 for c, h in zip(data['categories'], data['publisher_categories'])]
data['publisher_niche_count'] = data.groupby(['publisher', 'niche'])['name'].transform('count')
data['developer_niche_count'] = data.groupby(['developer', 'niche'])['name'].transform('count')
data['developer_std'] = data.groupby('developer')['price'].transform('std').fillna(0)
data['publisher_std'] = data.groupby('publisher')['price'].transform('std').fillna(0)

data['developer_price'] = data.groupby('developer')['price'].transform('mean')
data['publisher_price'] = data.groupby('publisher')['price'].transform('mean')
data['developer_recommendations'] = data.groupby('developer')['recommendations'].transform('mean')
data['publisher_recommendations'] = data.groupby('publisher')['recommendations'].transform('mean')
#data['quality'] = data['recommendations'] / (data['price'] + 0.001)
#data['developer_quality_ratio'] = data.groupby('developer')['quality'].transform('mean')
#data['publisher_quality_ratio'] = data.groupby('publisher')['quality'].transform('mean')
data['developer_momentum'] = data.copy().sort_values(['developer', 'release_date']).groupby('developer')['recommendations'].transform(lambda x: x.diff()).fillna(0)
#data['developer_niche_recommendations'] = data.groupby(['developer', 'niche'])['recommendations'].transform('mean')
data['publisher_momentum'] = data.copy().sort_values(['publisher', 'release_date']).groupby('publisher')['recommendations'].transform(lambda x: x.diff()).fillna(0)
#data['publisher_niche_recommendations'] = data.groupby(['publisher', 'niche'])['recommendations'].transform('mean')

In [ ]:
#data['log_developer_price'] = np.log1p(data['developer_price'])
data['log_publisher_price'] = np.log1p(data['publisher_price'])
data['log_developer_recommendations'] = np.log1p(data['developer_recommendations'])
data['log_publisher_recommendations'] = np.log1p(data['publisher_recommendations'])
#data['log_quality'] = np.log1p(data['quality'])
#data['log_developer_quality_ratio'] = np.log1p(data['developer_quality_ratio'])
#data['log_publisher_quality_ratio'] = np.log1p(data['publisher_quality_ratio'])
data['log_developer_momentum'] = np.sign(data['developer_momentum']) * np.log1p(np.abs(data['developer_momentum']))
#data['log_developer_niche_recommendations'] = np.log1p(data['developer_niche_recommendations'])
data['log_publisher_momentum'] = np.sign(data['publisher_momentum']) * np.log1p(np.abs(data['publisher_momentum']))
#data['log_publisher_niche_recommendations'] = np.log1p(data['publisher_niche_recommendations'])

data['developer_count'] = data['developer'].map(data['developer'].value_counts())
data['log_developer_count'] = np.log1p(data['developer_count'])
data['publisher_count'] = data['publisher'].map(data['publisher'].value_counts())
data['log_publisher_count'] = np.log1p(data['publisher_count'])

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
plot_data = data.explode('genres').explode('categories')
sns.barplot(data=plot_data, x='release_year', y='price')
plt.show()
sns.barplot(data=plot_data, x='release_year', y='recommendations')
plt.show()

In [ ]:
timeline_data = plot_data.groupby(['release_year', 'genres']).size().reset_index(name='count')
timeline_data_2 = plot_data.groupby(['release_year', 'categories']).size().reset_index(name='count')
sns.lineplot(data=timeline_data, x='release_year', y='count', hue='genres')
plt.show()
sns.lineplot(data=timeline_data_2, x='release_year', y='count', hue='categories')
plt.show()

In [ ]:
sns.barplot(data=plot_data, x='price', y='genres')
plt.show()
sns.barplot(data=plot_data, x='price', y='categories')
plt.show()
sns.barplot(data=plot_data, x='recommendations', y='genres')
plt.show()
sns.barplot(data=plot_data, x='recommendations', y='categories')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.histplot(data['price'], kde=True)
plt.show()
sns.histplot(data['recommendations'], kde=True)
plt.show()
sns.histplot(data['log_price'], kde=True)
plt.show()
sns.histplot(data['log_recommendations'], kde=True)
plt.show()